In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import seaborn as sns
from scipy import stats

In [2]:

chang_path="D:/中南财经政法大学/课程资料 class info/金融计量/参考数据"
os.chdir(chang_path)

In [3]:
dataframe1=pd.read_csv("CEO_Data.csv")
dataframe1['year'] = dataframe1.apply(lambda x : int(x['Reptdt'][0:4]),axis=1) 
dataframe1=dataframe1[['Stkcd', 'year', 'Female','Age','TotalSalary', 'IsDuality']]
dataframe1.rename(columns={"Stkcd": "stkcd"},inplace=True)
print(dataframe1.dtypes)

stkcd            int64
year             int64
Female           int64
Age            float64
TotalSalary    float64
IsDuality        int64
dtype: object


In [7]:
dataframe2=pd.read_csv("Female_board_per.csv")
dataframe3=pd.read_csv("公司年度价值指标.csv")
dataframe4=pd.read_csv("公司年度金融数据.csv")

In [8]:
whole_data=pd.merge(left=dataframe1,right=dataframe2, on=["stkcd","year"] ,how="inner") #默认是 inner交集，outer并集
whole_data=pd.merge(left=whole_data,right=dataframe3, on=["stkcd","year"] ,how="inner") #默认是 inner交集，outer并集
whole_data=pd.merge(left=whole_data,right=dataframe4, on=["stkcd","year"] ,how="inner") #默认是 inner交集，outer并集


In [9]:
whole_data.head()

,stkcd,year,Female,Age,TotalSalary,IsDuality,Female_board_per,Indirect_per,indcd,pe,tobinq,bm,asset,debt_asset,roa,log_asset
0,1,2006,0,64.0,9950000.0,1,0.000000,0.307692,J66,21.610182,1.060375,0.943159,2.605763e+11,0.975153,0.005396,26.286161
1,1,2007,0,65.0,22850000.0,1,0.071429,0.285714,J66,33.407078,1.164319,0.859036,3.525394e+11,0.963107,0.008641,26.588428
2,1,2007,0,59.0,4210000.0,0,0.071429,0.285714,J66,33.407078,1.164319,0.859036,3.525394e+11,0.963107,0.008641,26.588428
3,1,2008,0,66.0,15980000.0,1,0.071429,0.285714,J66,47.843208,1.024772,0.976061,4.744402e+11,0.965431,0.001485,26.885401
4,1,2008,0,60.0,4180000.0,0,0.071429,0.285714,J66,47.843208,1.024772,0.976061,4.744402e+11,0.965431,0.001485,26.885401


In [10]:
whole_data=whole_data[(whole_data.year<=2019) & (whole_data.year>=2006)]

In [11]:
for i in whole_data[['Age','TotalSalary','Female','Female_board_per', 'tobinq', 'asset', 'roa', 'debt_asset']]:
    whole_data[i]=np.where(whole_data[i].isnull(), np.nan, winsorize(np.ma.masked_invalid(whole_data[i]),limits=(0.01,0.01)))
    

In [13]:
df=whole_data[['TotalSalary','Female','Age','Female_board_per', 'tobinq', 'asset', 'roa', 'debt_asset']]
df["TotalSalary"] = (df["TotalSalary"]+1).apply(np.log1p)
df["asset"] = (df["asset"]+1).apply(np.log1p)


C:\Users\xuyonghao\AppData\Local\Temp\ipykernel_3664\1380839017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TotalSalary"] = (df["TotalSalary"]+1).apply(np.log1p)
C:\Users\xuyonghao\AppData\Local\Temp\ipykernel_3664\1380839017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["asset"] = (df["asset"]+1).apply(np.log1p)


In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


In [28]:
df=df.dropna()
df=add_constant(df)


In [29]:
vif = variance_inflation_factor(df.values,1)
vif=pd.Series([variance_inflation_factor(df.values, i) for i in range(df.shape[1])], index=df.columns)
print("VIF的结果如下\n",vif)


VIF的结果如下
 const               373.231505
TotalSalary           1.048334
Female                1.064106
Age                   1.035903
Female_board_per      1.078771
tobinq                1.164049
asset                 1.578698
roa                   1.296149
debt_asset            1.645953
dtype: float64


In [30]:
controls=df[['Female','Age','Female_board_per', 'tobinq', 'asset', 'roa', 'debt_asset']]
y = df['TotalSalary']

In [31]:
x1 = df[['Female','Age', 'tobinq', 'asset', 'roa', 'debt_asset']]
x1 = sm.add_constant(x1) 
y = df['TotalSalary']

In [32]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [33]:
result = sm.OLS(y, x1, missing="drop").fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            TotalSalary   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     279.1
Date:                Thu, 21 Sep 2023   Prob (F-statistic):               0.00
Time:                        09:20:18   Log-Likelihood:                -76050.
No. Observations:               35986   AIC:                         1.521e+05
Df Residuals:                   35979   BIC:                         1.522e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.8467      0.200     34.284      0.0

In [35]:
df_result = pd.concat((result.params, result.tvalues), axis=1)
df_result=df_result.rename(columns={0: 'beta', 1: 't'})
df_result


,beta,t
const,6.846670,34.283863
Female,0.153506,3.493908
Age,0.013939,8.406837
tobinq,0.005711,0.761219
asset,0.250160,27.211904
roa,3.045074,17.293804
debt_asset,-0.789516,-13.113701


In [36]:
with pd.ExcelWriter('OLS_Result_CEO.xlsx') as writer:
     df_result.to_excel(writer, sheet_name='sheet1')

In [1]:
myf_test = result.f_test('(Female = Age*10)')
print(myf_test)
#hypotheses = '(Female = Age)'
myf_test = result.f_test('(Female = Age)')
print(myf_test)

NameError: name 'result' is not defined